## Notebook containing code used for manuscript supplementary figure 6

### Note that most path variables throughout the notebook will need to be changed based on where the files were saved to your local folder

In [ ]:
import anndata
import numpy as np
import pandas as pd
import os
import sys

import plotly.graph_objects as go
import scipy

import spateo as st

In [ ]:
np.random.seed(888)

In [ ]:
%config InlineBackend.print_figure_kwargs={'dpi': 300.0}

## Resources for both the Midbrain-hindbrain boundary and spinal cord can be found here: https://www.dropbox.com/scl/fo/sxaqo4320hd16n183m6ph/ABzrK0DqPKcATdyPOId19M4?rlkey=v3vl90c8ct53upmfxtiflwnxo&st=q5itka64&dl=0

## Database files used here can be found: https://www.dropbox.com/scl/fo/dcd95so9zhkb8lnjkkxep/ANwmkFeb-sgtS89leHQezlU?rlkey=saiul4j5rr1vt6lwjl4hirmwh&st=brpjqw2c&dl=0

In [ ]:
# Set the Spateo database directory here:
database_dir = "/mnt/c/spateo-release-main/spateo/tools/database"

## Load MHB data object

In [ ]:
# Replace with wherever this file is stored locally
#path_to_mhb = "/mnt/d/SCData/Spateo_data/Stereo-seq_MHB/E11.5_MHB.h5ad"
path_to_mhb = "/mnt/d/Downloads/E11.5_MHB_final.h5ad"

In [ ]:
mhb = anndata.read_h5ad(path_to_mhb)

In [ ]:
# For analysis of spatial enrichment
mhb = st.tl.utils.create_new_coordinate(mhb, position_key="z_correction", plane="-xy")
mhb.obs["rc_coord"] = mhb.obs["-xy Coordinate"].copy()
del mhb.obs["-xy Coordinate"]
del mhb.uns

In [ ]:
# For analysis of spatial enrichment
mhb.write_h5ad(path_to_mhb)

In [ ]:
mhb.uns["__type"] = "UMI"

#### Initialize and run CCI model (can skip if predictions .csv file was created locally or downloaded from the folder)

In [ ]:
# Set to the folders that the inputs (ligands list, target genes list) are contained in and that the outputs (model results) will save to:
cci_input_directory = "/mnt/d/SCAnalysis/Spateo_MHB_analysis/CCI_inputs"
cci_output_directory = "/mnt/d/SCAnalysis/Spateo_MHB_analysis/CCI_outputs"
cci_output_id = os.path.join(cci_output_directory, "MHB_target_genes.csv")
cci_ligands_file = os.path.join(cci_input_directory, "mhb_ligands.txt")
cci_targets_file = os.path.join(cci_input_directory, "mhb_targets.txt")

In [ ]:
# Upper and lower bounds for the spatially-weighted regression model
lb = 6.42
ub = 15.97

In [ ]:
# This is how the above distance bounds are determined
lb = st.tl.find_neighbors.find_bw_for_n_neighbors(
    mhb,
    coords_key="z_correction",
    n_anchors=2000,
    target_n_neighbors=27,
    initial_bw=20,
    exclude_self=True
)
lb

In [ ]:
ub = st.tl.find_neighbors.find_bw_for_n_neighbors(
    mhb,
    coords_key="z_correction",
    n_anchors=2000,
    target_n_neighbors=270,
    initial_bw=20,
    exclude_self=True
)
ub

In [ ]:
adata_path = path_to_mhb
output_path = cci_output_id
ligand_path = cci_ligands_file
target_path = cci_targets_file
cci_dir_path = database_dir
mod_type = "ligand"
distr = "poisson"
species = "mouse"
group_key = "mapped_celltype"
coords_key = "z_correction"
distance_membrane_bound = lb
distance_secreted = ub
# Due to the scale of the dataset, quick fitting is not simple, incorporate spatial subsampling
spatial_subsample = True
total_counts_key = "total_counts"
total_counts_threshold = 2000.0
minbw = lb
maxbw = ub * 1.5

if not os.path.exists(os.path.dirname(output_path)):
    os.makedirs(os.path.dirname(output_path))

In [ ]:
parser, args_list = st.tl.define_spateo_argparse(
    adata_path=adata_path,
    custom_lig_path=ligand_path,
    targets_path=target_path,
    cci_dir=cci_dir_path,
    mod_type=mod_type,
    distr=distr,
    species=species,
    group_key=group_key,
    coords_key=coords_key,
    distance_membrane_bound=distance_membrane_bound,
    distance_secreted=distance_secreted,
    spatial_subsample=spatial_subsample,
    total_counts_key=total_counts_key,
    total_counts_threshold=total_counts_threshold,
    minbw=minbw,
    maxbw=maxbw,
    output_path=output_path,
)

In [ ]:
import time

t1 = time.time()

swr_model = st.tl.MuSIC(parser, args_list)
swr_model._set_up_model()
swr_model.fit()
swr_model.predict_and_save(adjust_for_subsampling=True)

t_last = time.time()

print("Total Time Elapsed:", np.round(t_last - t1, 2), "seconds")
print("-" * 60)

In [ ]:
# Use the same parser object to set up downstream analysis
downstream_model = st.tl.MuSIC_Interpreter(parser, args_list)

### Figure S6a- spatial enrichment of ligands and target genes

In [ ]:
# For clarity, visualize the created axis
mhb.obs["rc_coord"] = mhb.obs["-xy Coordinate"].copy()

axis = mhb.obs["rc_coord"]

x = mhb.obsm['z_correction'][:, 0]
y = mhb.obsm['z_correction'][:, 1]
z = mhb.obsm['z_correction'][:, 2]

fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=4,
        color=axis,  
        colorscale='Inferno',   
        colorbar=dict(title='R-C coordinate'),
    )
)])

title_dict = dict(
    text="Coordinate",
    y=0.9,
    yanchor="top",
    x=0.5,
    xanchor="center",
    font=dict(size=36),
)
fig.update_layout(
    scene=dict(
        aspectmode="data",
        xaxis=dict(
            showgrid=False,
            showline=False,
            linewidth=2,
            linecolor="black",
            backgroundcolor="white",
            title="",
            showticklabels=False,
            ticks="",
        ),
        yaxis=dict(
            showgrid=False,
            showline=False,
            linewidth=2,
            linecolor="black",
            backgroundcolor="white",
            title="",
            showticklabels=False,
            ticks="",
        ),
        zaxis=dict(
            showgrid=False,
            showline=False,
            linewidth=2,
            linecolor="black",
            backgroundcolor="white",
            title="",
            showticklabels=False,
            ticks="",
        ),
    ),
    margin=dict(l=0, r=0, b=0, t=50),  # Adjust margins to minimize spacing
    title=title_dict,
)

In [ ]:
# Visualize the enrichments of ligands and target genes along this axis- patterns coincident with Fgf8 are enriched in the midbrain-hindbrain boundary, to the anterior side there is enrichment in the midbrain,
# and the posterior side enrichment in the hindbrain
downstream_model.gene_expression_heatmap(
    use_ligands=True,
    position_key="rc_coord",
    neatly_arrange_y=True,
    window_size=35,
    cmap="bwr",
    title="E11.5 MHB- Ligand expression \ndistribution along rostral-caudal axis",
    fontsize=14,
)

In [ ]:
downstream_model.gene_expression_heatmap(
    use_target_genes=True,
    position_key="rc_coord",
    neatly_arrange_y=True,
    window_size=35,
    cmap="bwr",
    title="E11.5 MHB- Target gene expression \ndistribution along rostral-caudal axis",
    fontsize=14,
)

### Figure S6b- MHB inferred signaling effect heatmap (can skip this section if for_E11.5_MHB_heatmap file was downloaded)

In [ ]:
mhb = anndata.read_h5ad(path_to_mhb)
mhb.uns["__type"] = "UMI"

In [ ]:
def analyze_CCI_outputs(adata, gene_files_folder):
    results_dict = {}
    # Load the list of genes from the provided CSV files
    gene_files = os.listdir(gene_files_folder)
    
    for gene_file in tqdm(gene_files, desc="Processing all target genes..."):
        # Check and load only CSV files
        if not gene_file.endswith(".csv") or "predictions" in gene_file:
            continue
        
        # Extract the gene name from the file name
        gene_name = gene_file.split('_')[-1].split(".")[0]
        
        # Get the CCI results corresponding to this gene
        df = pd.read_csv(f"{gene_files_folder}/{gene_file}", index_col=0)

        for cell_type in ["Midbrain neuroectoderm", "Hindbrain neuroectoderm", "Midbrain-hindbrain boundary"]:
            type_cells = adata[adata.obs["mapped_celltype"] == cell_type].copy()
            # Get indices of cells of the current cell type
            indices = type_cells.obs.index
            
            # For each column in the DataFrame, calculate the proportion of non-zero entries for the current cell type
            proportions = (df.loc[indices] > 0).mean()
        
            # Store the proportions in the result dictionary
            for col in df.columns:
                results_key = f"{cell_type}-{col}"
                if results_key not in results_dict:
                    results_dict[results_key] = {}
                results_dict[results_key][gene_name] = proportions[col]
                
            # Clear intermediate large variables and suggest the GC to reclaim memory
            del type_cells, indices, proportions
            gc.collect()

        # Optionally clear the DataFrame read from file after processing
        del df
        gc.collect()
    
    # Convert results dictionary to DataFrame
    results_df = pd.DataFrame(results_dict)
    return results_df

In [ ]:
results = analyze_CCI_outputs(mhb, cci_output_directory)
results

In [ ]:
save_dir = "/mnt/d/SCAnalysis/Spateo_MHB_analysis"
results.to_csv(os.path.join(save_dir, "for_E11.5_MHB_heatmap.csv"))

In [ ]:
results = pd.read_csv(os.path.join(save_dir, "for_E11.5_MHB_heatmap.csv"), index_col=0)

In [ ]:
# Remove sets of columns if the ligand does not really have a large effect on any gene:
targets = set(col.split('-')[1] for col in results.columns)

# Iterate through each suffix
for target in targets:
    # Collect columns for the current suffix
    cols = [col for col in results.columns if col.endswith(target)]
    
    # Check if all values in these columns are < 0.1
    if results[cols].apply(lambda x: (x < 0.1).all()).all():
        # Remove these columns if the condition is met
        results.drop(columns=cols, inplace=True)
        
# Similarly, remove rows if no large effects are present:
results = results[(results.index == "Abcc4") | ~results.apply(lambda x: (x < 0.1).all(), axis=1)]
        
results = results[[c for c in results.columns if "se_" not in c]]
results

In [ ]:
# Normalize each row, scale for visuals
results = results.apply(lambda x: (x - x.min()) / (x.max() - x.min()), axis=1) ** 2
results

In [ ]:
results.to_csv(os.path.join(save_dir, "for_E11.5_MHB_heatmap.csv"))

#### Load heatmap file

In [ ]:
results = pd.read_csv(os.path.join(save_dir, "for_E11.5_MHB_heatmap.csv"), index_col=0)
# Mllt11 has many NaNs, do not include it:
results = results.loc[[i for i in results.index if i != "Mllt11"]]
results.columns = [c.replace("Midbrain-hindbrain boundary", "MHB NE") for c in results.columns]
results.columns = [c.replace("Midbrain neuroectoderm", "Midbrain NE") for c in results.columns]
results.columns = [c.replace("Hindbrain neuroectoderm", "Hindbrain NE") for c in results.columns]

In [ ]:
# Custom order for target genes:
targets = ["Cdc25b", "Fbxl17", "Helt", "Lrrc4c", "Pak3", "Tox", "Rgmb", "Il17rd", "Pam", "Akap6", "Ebf2", "Elavl4", "Fgd4", "Grid2", "Igsf8", "Nrxn1", "Rnd2", "Lrrn1", "Abcc4", "Igfbp5", "Rmst"]

In [ ]:
column_labels = results.columns.str.split('-', expand=True)
column_labels.columns = ['Group', 'Detail']
column_labels = column_labels.tolist()

# Reorder DataFrame columns with split labels
test = pd.MultiIndex.from_tuples(column_labels, names=['Group', 'Detail'])
results.columns = column_labels
results = results.loc[targets, :]

# Create a heatmap with labels
plt.figure(figsize=(24, 8))
vmin = 0
vmax = 0.8
mask = results < 0.1
cols_to_drop = mask.all(axis=0)
results = results.loc[:, ~cols_to_drop]
mask = results < 0.1

ax = sns.heatmap(
    results, 
    square=True,   
    linecolor="grey",
    linewidths=1, 
    cmap='magma',
    vmax=vmax,
    mask=mask,
)

cbar = ax.collections[0].colorbar
cbar.set_label('CCI effect score', size=20)
cbar.ax.tick_params(labelsize=20)

# Set the bottom ticks to show the 'Detail' and rotate them for better visibility
bottom_labels = [label[1].replace("b_", "") for label in results.columns]
ax.set_xticks(np.arange(len(bottom_labels)) + 0.5)
ax.set_xticklabels(bottom_labels, rotation=90, ha='center', fontsize=24)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=24)

In [ ]:
save_dir = "/mnt/d/SCAnalysis/Spateo_MHB_analysis"
plt.savefig(os.path.join(save_dir, "CCI_heatmap.svg"), format="svg", bbox_inches="tight")
plt.show()

### Figure S6c- Ptn effects plot

In [ ]:
def collect_b_Ptn_columns(directory_path):
    # Initialize an empty DataFrame to store the collected data
    collected_data = pd.DataFrame()

    # Iterate through all files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv') and '_' in filename:
            # Extract the id part from the filename
            file_id = filename.split('_')[-1].split('.')[0]

            # Read the 'b_Ptn' column from the file
            file_path = os.path.join(directory_path, filename)
            data = pd.read_csv(file_path, usecols=['b_Ptn'])

            # Rename the column to the extracted id and add it to the collected data
            data.rename(columns={'b_Ptn': file_id}, inplace=True)
            collected_data = pd.concat([collected_data, data], axis=1)

    return collected_data

In [ ]:
result_df = collect_b_Ptn_columns(cci_output_directory)
result_df

In [ ]:
# This subset are Ptn target genes
targets = ["Nrxn3", "Gap43", "Rtn1", "Akap6", "Mapk10", "Nrxn1", "Sorcs1", "Lrrc4c", "Pak3", "Pam", "Flrt3", "Arl4a", "Rgmb"]
result_df = result_df[targets]
result_df

In [ ]:
n_target_expr = (e115_mhb[:, targets].X.toarray() > 0).sum(axis=0)
n_target_expr

In [ ]:
proportions = (result_df > 0).sum() / n_target_expr
proportions

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))

top = proportions.sort_values(ascending=False)

# Create a vertical barplot with a line around each bar using the Series
sns.barplot(x=top.index, y=top.values, palette='RdPu', ax=ax, edgecolor='black')

# Rotate x-axis labels for better readability
plt.xticks(rotation=90, fontsize=14)

# Customize the plot
ax.set_title(f'Proportion Target-Expressing Cells Affected by Ptn', fontsize=16)
ax.set_xlabel('Gene', fontsize=14)
ax.set_ylabel('Proportion Gene-Expressing \nCells with Ptn Effect', fontsize=12)

# Show the plot
plt.show()

## For the next two sections- run CCI modeling + downstream modeling on the spinal cord interneuron subset

In [ ]:
# Set to the folders that the inputs (ligands list, target genes list) are contained in and that the outputs (model results) will save to:
cci_input_directory = "/mnt/d/SCAnalysis/Spateo_cord_analysis/CCI_inputs"
cci_output_directory = "/mnt/d/SCAnalysis/Spateo_cord_analysis/CCI_outputs"
cci_output_id = os.path.join(cci_output_directory, "cord_target_genes.csv")
cci_ligands_file = os.path.join(cci_input_directory, "cord_ligands.txt")
cci_targets_file = os.path.join(cci_input_directory, "cord_targets.txt")

In [ ]:
path_to_sc = "/mnt/d/Downloads/E11.5_spinal_cord.h5ad"
path_to_sc_interneuron = "/mnt/d/Downloads/E11.5_spinal_cord_interneurons.h5ad"

In [ ]:
# We use the full spinal cord to identify upper and lower bounds for the bandwidth
e115_sc = anndata.read_h5ad(path_to_sc)

In [ ]:
lb = st.tl.find_bw_for_n_neighbors(
    e115_sc,
    coords_key="z_correction",
    n_anchors=2000,
    target_n_neighbors=27,
    initial_bw=20,
    exclude_self=True
)

In [ ]:
ub = st.tl.find_bw_for_n_neighbors(
    e115_sc,
    coords_key="z_correction",
    n_anchors=2000,
    target_n_neighbors=270,
    initial_bw=20,
    exclude_self=True
)

In [ ]:
adata_path = path_to_sc_interneuron
output_path = cci_output_id
ligand_path = cci_ligands_file
target_path = cci_targets_file
cci_dir_path = database_dir
mod_type = "ligand"
distr = "poisson"
species = "mouse"
group_key = "mapped_celltype"
coords_key = "z_correction"
distance_membrane_bound = lb
distance_secreted = ub
spatial_subsample = True
total_counts_key = "total_counts"
total_counts_threshold = 3000.0
minbw = lb
maxbw = ub * 1.5

if not os.path.exists(os.path.dirname(output_path)):
    os.makedirs(os.path.dirname(output_path))

In [ ]:
parser, args_list = st.tl.define_spateo_argparse(
    adata_path=adata_path,
    custom_lig_path=ligand_path,
    custom_rec_path=receptor_path,
    targets_path=target_path,
    cci_dir=cci_dir_path,
    mod_type=mod_type,
    distr=distr,
    species=species,
    group_key=group_key,
    coords_key=coords_key,
    distance_membrane_bound=distance_membrane_bound,
    distance_secreted=distance_secreted,
    spatial_subsample=spatial_subsample,
    total_counts_key=total_counts_key,
    total_counts_threshold=total_counts_threshold,
    minbw=minbw,
    maxbw=maxbw,
    output_path=output_path,
)

In [ ]:
import time

t1 = time.time()

swr_model = st.tl.MuSIC(parser, args_list)
swr_model._set_up_model()
swr_model.fit()
swr_model.predict_and_save()

t_last = time.time()

print("Total Time Elapsed:", np.round(t_last - t1, 2), "seconds")
print("-" * 60)

In [ ]:
# Use the same parser object to set up downstream analysis
downstream_model = st.tl.MuSIC_Interpreter(parser, args_list)

In [ ]:
# Predict TF regulators of ligands
downstream_model.CCI_deg_detection_setup(
    group_key="mapped_celltype",
    use_ligands=True,
    use_receptors=False,
    use_targets=False
)

In [ ]:
downstream_model.CCI_deg_detection(
    group_key="mapped_celltype",
    cci_dir_path=cci_dir_path,
    use_ligands=True,
    use_receptors=False,
    use_targets=False,
    use_dim_reduction=False,
    distr="poisson"
)

### Figure S6k- predicted regulators of Slit2

In [ ]:
exclude = [
    "Rarb",
    "Pax2",
    "Rara",
    "Ctcf",
    "Zfp281",
    "Isl1",
    "Zfp652"
]

In [ ]:
interaction_subset = [f.replace("regulator_", "") for f in downstream_model.downstream_model_ligand_design_matrix.columns if f.replace("regulator_", "") not in exclude]

In [ ]:
downstream_model.deg_effect_barplot(
    target="Slit2",
    interaction_subset=interaction_subset,
    top_n_interactions=25,
    figsize=(5, 1.5)
)

### Figure S6l- predicted targets of Slit2

In [ ]:
downstream_model.top_target_barplot(
    interaction="Slit2",
    figsize=(6, 2),
    top_n_targets=40
)